515: function to check if there is a subset with sum divisible by m

In [ ]:
%%writefile bugged_modular.py
def modular_sum(arr, n, m):
    if (n > m):
        return True

    DP = [False for i in range(m)]

    for i in range(n):
        if (DP[1]):                    ## Bug: changed DP[0] to DP[1]
            return True
        temp = [False for i in range(m)]

        for j in range(m):
            if (DP[j] == True):
                if (DP[(j + arr[i]) % m] == False):
                    temp[(j + arr[i]) % m] = True

        for j in range(m):
            if (temp[j]):
                DP[j] = True
                
        DP[arr[i] % m] = True

    return DP[0]

Overwriting bugged_modular.py


In [7]:
%%writefile llm_test_modular.py

from bugged_modular import modular_sum


def test_single_element_divisible():
    arr = [5]
    n = 1
    m = 5
    # 5 % 5 == 0
    assert modular_sum(arr, n, m) is True


def test_single_element_not_divisible():
    arr = [3]
    n = 1
    m = 5
    assert modular_sum(arr, n, m) is False


def test_empty_array_no_subset():
    # Assuming non-empty subset requirement
    arr = []
    n = 0
    m = 7
    assert modular_sum(arr, n, m) is False


def test_all_zeros_always_divisible():
    # Any non-empty subset of zeros has sum 0 → divisible by any m
    arr = [0, 0, 0]
    n = 3
    m = 3
    assert modular_sum(arr, n, m) is True


def test_m_equals_one_always_true():
    # Any integer is divisible by 1
    arr = [7, 11, 13]
    n = 3
    m = 1
    assert modular_sum(arr, n, m) is True


def test_typical_true_subset():
    # 1 + 5 = 6, divisible by 6
    arr = [3, 1, 7, 5]
    n = 4
    m = 6
    assert modular_sum(arr, n, m) is True


def test_typical_false_subset():
    # No subset sums of [2,3,4] are divisible by 8
    arr = [2, 3, 4]
    n = 3
    m = 8
    assert modular_sum(arr, n, m) is False


def test_negative_numbers_allowed():
    # -1 + 2 + 3 = 4 → divisible by 4
    arr = [-1, 2, 3]
    n = 3
    m = 4
    assert modular_sum(arr, n, m) is True


def test_large_numbers_divisible():
    arr = [100_000_000, 200_000_000]
    n = 2
    m = 100
    assert modular_sum(arr, n, m) is True


def test_repeated_values_combination():
    # 4 + 4 + 2 = 10 → divisible by 10
    arr = [4, 4, 2]
    n = 3
    m = 10
    assert modular_sum(arr, n, m) is True


def test_all_same_not_divisible():
    # sums: 3, 6, 9 → none divisible by 7
    arr = [3, 3, 3]
    n = 3
    m = 7
    assert modular_sum(arr, n, m) is False


def test_minimal_pair_true():
    arr = [2, 3]
    n = 2
    m = 5
    # 2 + 3 = 5
    assert modular_sum(arr, n, m) is True


Overwriting llm_test_modular.py


In [8]:
!pytest -v llm_test_modular.py

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-8.4.2, pluggy-1.6.0 -- C:\Python313\python.exe
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: d:\IIIT HYD\SSD\Final project\HumanVsLLM\515
plugins: anyio-4.9.0, hypothesis-6.147.0
collecting ... collected 12 items

llm_test_modular.py::test_single_element_divisible PASSED                [  8%]
llm_test_modular.py::test_single_element_not_divisible PASSED            [ 16%]
llm_test_modular.py::test_empty_array_no_subset PASSED                   [ 25%]
llm_test_modular.py::test_all_zeros_always_divisible PASSED              [ 33%]
llm_test_modular.py::test_m_equals_one_always_true PASSED                [ 41%]
llm_test_modular.py::test_typical_true_subset PASSED                     [ 50%]
llm_test_modular.py::test_typical_false_subset PASSED                    [ 58%]
llm_test_modular.py::test_negative_numbers_allowed PASSED                [ 66%]
llm_test_m

In [9]:
%%writefile human_test_modular.py

from hypothesis import given, strategies as st, assume
from bugged_modular import modular_sum


@given(st.lists(st.integers(min_value=1, max_value=50), min_size=1, max_size=15),
       st.integers(min_value=2, max_value=50))
def test_property_multiply(arr, m):
    """Multiplying all elements by m should always give True"""
    scaled_arr = [x * m for x in arr]
    n = len(scaled_arr)
    result = modular_sum(scaled_arr, n, m)
    
    assert result is True


@given(st.lists(st.integers(min_value=-100, max_value=100), min_size=1, max_size=15),
       st.integers(min_value=2, max_value=50))
def test_property_reverse(arr, m):
    """Result should be independent of element ordering"""
    n = len(arr)
    result_original = modular_sum(arr, n, m)
    result_reversed = modular_sum(arr[::-1], n, m)
    
    assert result_original == result_reversed


@given(st.lists(st.integers(min_value=-100, max_value=100), min_size=1, max_size=10),
       st.integers(min_value=2, max_value=50))
def test_property_idempotence(arr, m):
    """If solution exists, it still exists when we duplicate the array"""
    n = len(arr)
    result_original = modular_sum(arr, n, m)
    
    arr_doubled = arr + arr
    result_doubled = modular_sum(arr_doubled, len(arr_doubled), m)
    
    # True should stay True
    if result_original:
        assert result_doubled is True


@given(st.lists(st.integers(min_value=-100, max_value=100), min_size=1, max_size=10),
       st.integers(min_value=-100, max_value=100),
       st.integers(min_value=2, max_value=50))
def test_property_monotonicity(arr, new_element, m):
    """Adding an element to the array can only increase solution possibilities"""
    n = len(arr)
    result_before = modular_sum(arr, n, m)
    
    arr_after = arr + [new_element]
    result_after = modular_sum(arr_after, len(arr_after), m)
    
    # if solution is there, it will be after too
    if result_before:
        assert result_after is True


@given(st.lists(st.integers(min_value=-50, max_value=50), min_size=1, max_size=8),
       st.integers(min_value=2, max_value=30))
def test_property_brute_force(arr, m):
    """For small arrays, verify against exhaustive subset enumeration"""
    assume(len(arr) <= 8)
    
    n = len(arr)
    result = modular_sum(arr, n, m)
    
    # Brute force: check all non-empty subsets
    valid_subset = False
    for i in range(1, 2**n):
        subset_sum = sum(arr[j] for j in range(n) if i & (1 << j))
        if subset_sum % m == 0:
            valid_subset = True
            break
    
    assert result == valid_subset

Overwriting human_test_modular.py


In [10]:
!pytest -v human_test_modular.py

============================= test session starts =============================
platform win32 -- Python 3.13.2, pytest-8.4.2, pluggy-1.6.0 -- C:\Python313\python.exe
cachedir: .pytest_cache
hypothesis profile 'default'
rootdir: d:\IIIT HYD\SSD\Final project\HumanVsLLM\515
plugins: anyio-4.9.0, hypothesis-6.147.0
collecting ... collected 5 items

human_test_modular.py::test_property_multiply PASSED                     [ 20%]
human_test_modular.py::test_property_reverse FAILED                      [ 40%]
human_test_modular.py::test_property_idempotence PASSED                  [ 60%]
human_test_modular.py::test_property_monotonicity PASSED                 [ 80%]
human_test_modular.py::test_property_brute_force FAILED                  [100%]

================================== FAILURES ===================================
____________________________ test_property_reverse ____________________________

    @given(st.lists(st.integers(min_value=-100, max_value=100), min_size=1, max_size=15),